In [1]:
import pandas as pd
import datetime as dt
import time
import numpy as np
from functools import reduce

In [2]:
def csv_to_dict(path):
    df = pd.read_csv(path)
    df.rename(columns = {df.columns[0]:'DATE', df.columns[1]: '{}_'.format(path[:-4]) + 'VALUE'},inplace=True)
    df['DATE'] = pd.to_datetime(df['DATE'])
    str_date = [str(x) for x in df['DATE']]
    dt_list = [dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in str_date]
    value_list = [x for x in df.iloc[:,1]]
    dict_df = {'DATE':dt_list,df.columns[1]:value_list}
    return dict_df

In [3]:
n = csv_to_dict('NKY.csv')
h = csv_to_dict('HSI.csv')
s = csv_to_dict('SPX.csv')

In [13]:
def clean(option: str, *args: dict):
    d = []
    q = []
    for arg in args:
        d.append(pd.DataFrame.from_dict(arg))

    l = pd.date_range(start = min([i['DATE'].iloc[0] for i in d]), end = max([i['DATE'].iloc[-1] for i in d]))
    a = pd.DataFrame(l,columns=['DATE'])
    a['VALUE'] = np.full(len(a),np.nan)

    d.insert(0,a)
    df_merged = reduce(lambda left, right: pd.merge(left, right, on=['DATE'],how='outer'), d)
    for j in range(len(args)):
        q.extend([i for i in list(args[j].keys()) if 'VALUE' in i])
    renamed_col = ['DATE'] + ['NaN'] + q
    df_merged.columns = renamed_col
    df_merged = df_merged.drop(df_merged.columns[1],axis=1)
    
    if option == 'FLATRIGHT':
        df_merged = df_merged.fillna(method='ffill').round(4)
        
    if option == 'LINEAR':
        df_merged = df_merged.interpolate(method='linear',axis=0).round(4)

    if option == 'REMOVE':
        df_merged = df_merged.dropna()
    
        
    return df_merged    

In [15]:
clean('REMOVE',n,h,s).head()

,DATE,NKY_VALUE,HSI_VALUE,SPX_VALUE
0,2017-09-22,20296.45,27880.53,2502.22
3,2017-09-25,20397.58,27500.34,2496.66
4,2017-09-26,20330.19,27513.01,2496.84
5,2017-09-27,20267.05,27642.43,2507.04
6,2017-09-28,20363.11,27421.60,2510.06


In [16]:
clean('LINEAR',n,h,s).head()

,DATE,NKY_VALUE,HSI_VALUE,SPX_VALUE
0,2017-09-22,20296.45,27880.53,2502.2200
1,2017-09-23,20330.16,27753.80,2500.3667
2,2017-09-24,20363.87,27627.07,2498.5133
3,2017-09-25,20397.58,27500.34,2496.6600
4,2017-09-26,20330.19,27513.01,2496.8400


In [17]:
clean('FLATRIGHT',n,h,s).head()

,DATE,NKY_VALUE,HSI_VALUE,SPX_VALUE
0,2017-09-22,20296.45,27880.53,2502.22
1,2017-09-23,20296.45,27880.53,2502.22
2,2017-09-24,20296.45,27880.53,2502.22
3,2017-09-25,20397.58,27500.34,2496.66
4,2017-09-26,20330.19,27513.01,2496.84
